# Data Visualisation and Exploration 

Import required modules and custom functions for visualisations. Initialize the SQLite connection 

In [1]:
import sqlite3
import pandas as pd

from lets_plot import *
LetsPlot.setup_html()

import visualizations

## 1. Read Data from SQLite Database
I read in the data from topcharts.db and joined tables so that I had complete information on the (track_id,trending_year) level.

In [3]:
with sqlite3.connect('../data/topcharts.db') as connection:
    artist_genres_df = pd.read_sql_query("SELECT * FROM artist_genres", connection)
    artists_df = pd.read_sql_query("SELECT * FROM artists", connection)
    charts_df = pd.read_sql_query("SELECT * FROM charts", connection)
    songs_df = pd.read_sql_query("SELECT * FROM songs", connection)
    genres_df = pd.read_sql_query("SELECT * FROM genres", connection)
    unemployment_df = pd.read_sql_query("SELECT * FROM unemployment", connection)

full_df = pd.merge(artists_df, artist_genres_df, on='artist_id', how='left').rename(columns={'genre': 'sub_genre'})
full_df = pd.merge(full_df, genres_df, on='sub_genre', how='left')
full_df = pd.merge(songs_df, full_df, on='artist_id', how='left')
full_df = pd.merge(full_df, charts_df, on='track_id', how='left')
full_df['year'] = full_df['trending_year'].astype(int)
full_df = pd.merge(full_df, unemployment_df, on='year', how='left')
full_df = full_df.drop(columns=['trending_year'])

## 1. Exploring General Trends 
First, I looked at the trends in the data in general. I wanted to see which genres and artists appeared most frequently, the extend to which it appeared over time, etc before exploring the impact of economic periods. 

In [ ]:
table1 = visualizations.get_mean_genre_appearance_table(full_df)

### Table 1: EDM and Pop Appear Most Frequently in the Charts
#### Mean Number of Appearences in the UK top 40 tracks each year
| Genre                   |   Mean Appearances |
|:------------------------|-------------------:|
| EDM, House, and Dance   |              98.38 |
| Pop                     |              73.96 |
| Rap and Hip Hop         |              35.33 |
| Rock                    |              11.1  |
| R&B                     |               7.85 |
| Indie                   |               4.83 |
| Movies, TV, and Theater |               4.1  |
| Latin Inspired          |               3.5  |
| Jazz and Soul           |               3.35 |
| Country and Folk        |               2    |

*the number of appearences can exceed 40 (although these are the top 40 charts) because songs can have multiple genres*

This table indicates that the biggest genres are EDM/House and Pop, followed by Rap and Rock. Other genres have less frequent / more minor appearences in the charts. 

In [ ]:
table2 = visualizations.get_top_genre_table(full_df)

### Table 2: Top Genre shifts Between EDM, Pop, and Rap 
#### Top Genre in the UK top 40 tracks each year

|   year | genre                 |   appearances |
|-------:|:----------------------|--------------:|
|   2000 | EDM, House, and Dance |            47 |
|   2001 | Pop                   |            34 |
|   2002 | Pop                   |            50 |
|   2003 | Rap and Hip Hop       |            59 |
|   2004 | Rap and Hip Hop       |            69 |
|   2005 | Rap and Hip Hop       |            51 |
|   2006 | Pop                   |            49 |
|   2007 | Pop                   |            59 |
|   2008 | Pop                   |            71 |
|   2009 | EDM, House, and Dance |           194 |
|   2010 | Pop                   |            79 |
|   2011 | EDM, House, and Dance |           173 |
|   2012 | Pop                   |            76 |
|   2013 | EDM, House, and Dance |           290 |
|   2014 | EDM, House, and Dance |           135 |
|   2015 | EDM, House, and Dance |           183 |
|   2016 | EDM, House, and Dance |           106 |
|   2017 | EDM, House, and Dance |           227 |
|   2018 | EDM, House, and Dance |           228 |
|   2019 | EDM, House, and Dance |           117 |
|   2020 | Rap and Hip Hop       |            46 |
|   2021 | EDM, House, and Dance |            86 |
|   2022 | EDM, House, and Dance |            96 |
|   2023 | EDM, House, and Dance |           150 |

*the number of appearences can exceed 40 (although these are the top 40 charts) because songs can have multiple genres*

This table shows that the most frequently occuring genre of the year in the UK top charts is always either Pop, EDM, or Rap. It is interesting that genres seem to be clustered together; with Pop and Rap dominating the early 2000s and EDM dominating after 2013 (but emerging in prominence in 2009). Additionally the re-emergence of Rap in 2020 is interesting, particularly as I look at the relationship between economics and music listening, as 2020 was an economically unique year due to Covid-19. 


In [4]:
genre_trends_barplot = visualizations.plot_genre_trends(full_df, 'barchart')

### Figure 3:

In [5]:
genre_trends_barplot

Pop music slightly rises and falls from year to year but generally maintains a share of the chart between 25%-40% and doesn't show much significant change from 2000-2023. Rap and Hip Hop shows a sharp decrease from 2009 until 2020, which is interesting, and potentially could align with the theory that resession alters music consumption, althogh the causality is not clear. Similarly, EDM had a sharp increase in 2009 that it has maintained relatively constantly through 2023. Again, this follows the theory that the recession impacted consumption, increasing club/party music, which is typically theorized to be popular 'recession music'. This also shows that more 'minor' genres (rock, indie, R&B) were more popular until 2008, where they decreased in popularity. 

Overall, although there is no causality reflected in this graph, it is interesting that there does seem to be a partial shift in 2008-2009 (potentially in line with the recession) where EDM rose in populraity, and Rap and other, less popular genres fell. 

## 2. Exploring Trends During Different Economic Periods
Next, I looked at whether or not Genre popularity changed during different economic periods in the UK 

### Figure 4: EDM and Pop are more Popular During Recessions

In [ ]:
p = visualizations.plot_single_genre_proportion(full_df, 'Recession')

#### Figure 4.1: EDM is Popular During Recessions, Minor Genres are Very Unpopular

In [6]:
p.show()

In [ ]:
p = visualizations.plot_single_genre_proportion(full_df, 'Moderate Unemployment')

#### Figure 4.2: EDM and Pop are Popular During Moderate Economic Periods, Minor Genres are Somewhat Popular

In [7]:
p.show()

In [ ]:
p = visualizations.plot_single_genre_proportion(full_df, 'Economic Stability')

#### Figure 4.3: EDM, Pop, and Rap are Popular During Moderate Economic Periods; Minor Genres are Somewhat Popular

In [8]:
p.show()

The three plots above (Figure 4) show interesting differences between economic periods. During economic recessions (4.1), only the 3 most popular genres (Pop, EDM, Rap) have significant shares of the charts. EDM is very popular, pop is less but still very popular, rap is less popular than normal, and genres like country and latin do not appear at all.
During moderate economic periods (4.2), Pop and EDM share relatively equal popularities. Rap is more popular than during recessions. All minor genres are represented in the charts but with minor shares.
During stable economic periods (4.3), Pop and EDM still dominate the charts 

## 3. Exploring Economic Trends and Minor Genre Popularity

### Figure 5:

In [5]:
plot = visualizations.plot_minor_genres(full_df)

In [7]:
plot.show()

Zooming in on the minor genres (not pop, edm, or rap) shows a general decline in their populariy since 2000. If music listening is related to economic conditios, it seems that the 2008 recession was correlated with a drop in popularity of minor genres, while the covid-19 (more minor recession) was correlated with a rise in popularity of minor genres. Its unclear why there would be different music consumption for the different recessions, but it does appear that those time periods are the turning point. It is also generally interesting to see that minor genres are decreasing in popularity as the charts are becoming more dominated by the same 3 genres. 

## 4. Conclusions
 
 


This analysis provides insights into the interplay between genre popularity in the UK Top 40 charts and economic conditions over the past two decades. Key conclusions include:

1. *Dominance of Major Genres:*

- Pop, EDM, and Rap consistently dominate the charts, with EDM emerging as the most frequent genre after 2009.
- Minor genres (Rock, Indie, R&B, etc.) have seen a steady decline in popularity, suggesting a shift in listener preferences or industry trends toward mainstream sounds.

2. *Impact of Economic Conditions:*

- The 2008 recession coincided with a significant rise in EDM's popularity and a decline in both Rap and minor genres, aligning with theories of recession music favoring pop, high-energy, and club sounds. Although causality can't be established from this data, it does support previous theories of recession pop and recession music generally. 
- During periods of economic stability, there is greater diversity in genre representation, although Pop and EDM remain dominant.
- The COVID-19 pandemic in 2020 saw a resurgence in Rap and a temporary increase in minor genre popularity, hinting at differing impacts of economic uncertainty based on its nature and context.

3. *Trends in Minor Genres:*

- Minor genres show a long-term decline, particularly after 2008, with their minimal representation during recessions. This trend may reflect a consolidation of music preferences around the most popular genres during challenging economic periods.